In [39]:
# dataset
FILENAME = "compounds_ms2structures"

# similarity calculation
MORGAN_RADIUS=9
FINGERPRINT_BITS=4096

# analogue selection
NUM_QUERIES = 1000
GROUP_SIZE=10
SIMILARITY_RANGE=(0.7, 0.9999)
NO_OVERLAP=False
RANDOM_SELECT = True   # Set to false to iterate through the dataset sequentially. Might be be better for high similarity selections.
SEED=42

# unsimilar query selection
MAX_SIMILARITY = 0.15

## Init

In [ ]:
import os
import random
random.seed(SEED)
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from rdkit import DataStructs
from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import SmilesMolSupplier

from ms_chemical_space_explorer.similarity_maps import get_similarity_map_weights, FingerprintFunction

path = os.path.join( "data", "datasets", FILENAME + ".csv")
fingerprints_file = os.path.join("data", "group_similarity", f"{FILENAME}_fingerprints_morgan{MORGAN_RADIUS}_{FINGERPRINT_BITS}bits.npy")
sim_matrix_file = os.path.join("data", "group_similarity", f"{FILENAME}_ruzicka_similarities_morgan{MORGAN_RADIUS}_{FINGERPRINT_BITS}bits.npy")
df_file = os.path.join("data", "group_similarity", "benchmarks", f"{FILENAME}_benchmark_num_queries{NUM_QUERIES}_random{int(RANDOM_SELECT)}_size{GROUP_SIZE}_range{SIMILARITY_RANGE}_no_overlap{int(NO_OVERLAP)}_seed{SEED}_morgan{MORGAN_RADIUS}_{FINGERPRINT_BITS}bits.csv")

if FILENAME == "compounds_ms2structures":
    mol_supplier = SmilesMolSupplier(path, delimiter=',', titleLine=True, smilesColumn=1, nameColumn=0)
elif FILENAME == "biostructures_combined":
    mol_supplier = SmilesMolSupplier(path, titleLine=True, nameColumn=0)

# compounds = pd.read_csv(path)
# compounds.head()

## Fingerprints & Similarities

In [41]:
fingerprints = np.load(fingerprints_file, mmap_mode ='r')
sim_matrix = np.load(sim_matrix_file, mmap_mode ='r')

print(fingerprints.shape, sim_matrix.shape)

(37811, 4096) (37811, 37811)


## Load benchmark with similar groups

In [42]:
def str_to_list_of_ints(series):
    return [int(x) for x in series.strip("[]").split(",") if x.strip().isdigit()]


analogue_df = pd.read_csv(df_file, index_col=0)
analogue_df["analogue_ids"] = analogue_df["analogue_ids"].apply(str_to_list_of_ints)
print(f"Found {df_file}\nLoaded {len(analogue_df)} analogue groups.")
analogue_df

Found data\group_similarity\benchmarks\compounds_ms2structures_benchmark_num_queries1000_random1_size10_range(0.7, 0.9999)_no_overlap0_seed42_morgan9_4096bits.csv
Loaded 1000 analogue groups.


,analogue_ids,query_sim,group_sim,sgs,sgs_score
query_id,,,,,
35713,"[23200, 15235, 23492, 16509, 788, 36440, 6612,...",0.757227,0.651649,0.718469,0.958369
6698,"[4956, 18407, 36935, 14442, 34068, 12344, 9297...",0.728808,0.741461,0.711796,0.930022
23723,"[6469, 4900, 21458, 21338, 4110, 2985, 4580, 3...",0.776946,0.714194,0.768828,0.958988
23219,"[34068, 19889, 23098, 30328, 4192, 27915, 4203...",0.764761,0.679330,0.671523,0.952904
7161,"[6420, 34482, 15048, 9393, 8816, 6018, 14039, ...",0.752489,0.651582,0.704055,0.949291
...,...,...,...,...,...
9389,"[13371, 16668, 1846, 4964, 12, 26636, 25711, 3...",0.856211,0.793353,0.725266,0.981181
34088,"[16501, 19951, 7447, 20032, 33526, 11214, 2076...",0.793224,0.726373,0.781378,0.950357
26225,"[8055, 31010, 7549, 29135, 276, 7567, 27413, 3...",0.754275,0.703045,0.700822,0.937447


## Benchmark

In [5]:
analogue_df.analogue_ids[35713]

[23200, 15235, 23492, 16509, 788, 36440, 6612, 29122, 12150, 18256]

In [6]:
def pick_unsimilar_query(group_ids, sim_matrix, max_similarity_threshold):
    sims =[]
    used_queries = []
    
    query_id = random.randint(0, len(sim_matrix) - 1)
    found_unsimilar_query = False
    while found_unsimilar_query == False:
        # pick unused query
        while query_id in used_queries:
            query_id = random.randint(0, len(sim_matrix) - 1)
        used_queries.append(query_id)
        
        # get similarities to group
        sims = []
        for group_id in group_ids:
            sims.append(sim_matrix[query_id, group_id])
        
        # check if query is unsimilar enough
        if np.max(sims) < max_similarity_threshold:
            break
    
    return query_id, sims

In [ ]:
def get_unsimilar_benchmark(df_file, analogue_df, sim_matrix, mol_supplier, morgan_radius, fpSize,  max_similarity_threshold):
    try:
        print(f"Trying to load benchmark dataframe from {df_file}")
        analogue_df = pd.read_csv(df_file, index_col=0)
        analogue_df["analogue_ids"] = analogue_df["analogue_ids"].apply(str_to_list_of_ints)
        print(f"Found {df_file}\nTrying to access benchmark columns.\nExisting columns: {analogue_df.columns.tolist()}")
        query_sim_unsimilar= analogue_df[:1].query_sim_unsimilar
        sgs_score_unsimilar=analogue_df[:1].sgs_score_unsimilar
        print(f"Benchmark found! Loaded benchmark for {len(analogue_df)} queries.")
    except Exception as e:
        print(f"Benchmark not found: {df_file}\nRunning the benchmark dataframe generation.")
        # generate benchmark dataframe
        fp_function = FingerprintFunction(fingerprint="MORGAN")
        fp_function.fp_type = "count"
        fp_function.morgan_radius = morgan_radius
        fp_function.nbits = fpSize

        for query_id, row in tqdm(analogue_df.iterrows(), desc="Processing queries", total=len(analogue_df)):
            analogue_ids = [int(x) for x in row["analogue_ids"]]
            query_id_unsimilar, query_sims_unsimilar = pick_unsimilar_query(analogue_ids, sim_matrix, max_similarity_threshold)
            analogue_df.loc[query_id, 'query_id_unsimilar'] = query_id_unsimilar
            analogue_df.loc[query_id, 'query_sim_unsimilar'] = np.mean(query_sims_unsimilar)

            query = mol_supplier[query_id_unsimilar]
            
            # query_sims = []
            stacked_mean_weights = []
            query_scaled_inverted_differnces = []
            for ref_id in tqdm(analogue_ids, desc=f"Processing analogues for query {query_id_unsimilar}", leave=False):
                # # gather ruzicka similarity of analogue vs query
                # query_sims.append(sim_matrix[query_id_unsimilar][ref_id])
                
                # generate the reference molecule
                ref_analogue = mol_supplier[ref_id]
                
                # create array for stacked weights
                stacked_atomic_weights = [0] * ref_analogue.GetNumAtoms()
                
                # compute the group similarity map weights
                for probe_id in analogue_ids:#tqdm(analogue_ids, desc=f"Computing weights for weights for analogue {ref_id}", leave=False):
                    if ref_id == probe_id:
                        continue
                    
                    probe_analogue = mol_supplier[probe_id]

                    # extract and standardize similarity weights for the reference molecule
                    ref_sim_weights = SimilarityMaps.GetAtomicWeightsForFingerprint(probe_analogue, ref_analogue, fp_function.get_function, metric=DataStructs.TanimotoSimilarity) #this is the bottle neck
                    #print(f"Getting weights took {(pd.Timestamp.now() - getting_weights_start).total_seconds():.4f} seconds")

                    # stack the weights
                    stacked_atomic_weights = [stacked_atomic_weights[i] + ref_sim_weights[i] for i in range(ref_analogue.GetNumAtoms())]
                # Standardize the stacked weights
                stacked_atomic_weights, _ = SimilarityMaps.GetStandardizedWeights(stacked_atomic_weights)
                stacked_mean_weights.append(sum(stacked_atomic_weights) / len(stacked_atomic_weights))
                
                # compute the similarity map weights against the query molecule
                query_atomic_weights = get_similarity_map_weights(ref_analogue, query, fp_function)
                
                # calculate absolute weight difference for each atom against the query
                query_abs_differences = ([abs(stacked_atomic_weights[i] - query_atomic_weights[i]) for i in range(len(stacked_atomic_weights))])
                query_mean_difference = sum(query_abs_differences) / len(query_abs_differences)
                
                # scale and invert the mean absolute difference to get a similarity score
                query_scaled_inverted_differnce = 1 - (query_mean_difference * 0.5)
                query_scaled_inverted_differnces.append(query_scaled_inverted_differnce)
            # --- gather all similarity metrics ---
            # # mean ruzicka similarity of analogue vs query ruzicka similaities
            # analogue_df.loc[query_id, "query_sim"] = np.mean(query_sims)
            
            # # mean of all-vs-all analogue ruzicka similarities
            # fp_group_sim_matrix = ruzicka_similarity_matrix(fingerprints[analogue_ids], fingerprints[analogue_ids])
            # group_sim = off_diagonal_mean(fp_group_sim_matrix)
            # analogue_df.loc[query_id, "group_sim"] = group_sim
            
            # mean of the means of stacked atomic weights of all-vs-all analogue similarity maps
            unscaled_sgs = np.mean(stacked_mean_weights)                # range [-1, 1]
            analogue_df.loc[query_id, "sgs_unsimilar"] = (unscaled_sgs + 1) / 2   # range [0, 1]
            
            # scaled and inverted mean of the means of the absolute differences between atomic stacked weights and atomic query weights
            sgs_score = (sum(query_scaled_inverted_differnces) / len(query_scaled_inverted_differnces))
            analogue_df.loc[query_id, "sgs_score_unsimilar"] = sgs_score

        analogue_df.to_csv(df_file)
        print(f"Saved {len(analogue_df)} queries to {df_file}.")
    return analogue_df

In [ ]:
analogue_df = get_unsimilar_benchmark(df_file, analogue_df, sim_matrix, mol_supplier, MORGAN_RADIUS, FINGERPRINT_BITS, MAX_SIMILARITY)

Found data\group_similarity\benchmarks\compounds_ms2structures_benchmark_num_queries1000_random1_size10_range(0.7, 0.9999)_no_overlap0_seed42_morgan9_4096bits.csv
Trying to access benchmark columns.
Existing columns: ['analogue_ids', 'query_sim', 'group_sim', 'sgs', 'sgs_score']
Benchmark not found: data\group_similarity\benchmarks\compounds_ms2structures_benchmark_num_queries1000_random1_size10_range(0.7, 0.9999)_no_overlap0_seed42_morgan9_4096bits.csv
Running the benchmark dataframe generation.


Processing queries:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing analogues for query 7296:   0%|          | 0/10 [00:00<?, ?it/s]

Processing analogues for query 1639:   0%|          | 0/10 [00:00<?, ?it/s]

Processing analogues for query 18024:   0%|          | 0/10 [00:00<?, ?it/s]

...

Processing analogues for query 17181:   0%|          | 0/10 [00:00<?, ?it/s]

Processing analogues for query 24970:   0%|          | 0/10 [00:00<?, ?it/s]

Processing analogues for query 24344:   0%|          | 0/10 [00:00<?, ?it/s]

Saved 1979 queries to data\group_similarity\benchmarks\compounds_ms2structures_benchmark_num_queries1000_random1_size10_range(0.7, 0.9999)_no_overlap0_seed42_morgan9_4096bits.csv.


,analogue_ids,query_sim,group_sim,sgs,sgs_score,query_id_unsimilar,query_sim_unsimilar,sgs_unsimilar,sgs_score_unsimilar
query_id,,,,,,,,,
35713,"[23200, 15235, 23492, 16509, 788, 36440, 6612,...",0.757227,0.651649,0.718469,0.958369,7296.0,0.048941,NaN,NaN
6698,"[4956, 18407, 36935, 14442, 34068, 12344, 9297...",0.728808,0.741461,0.711796,0.930022,1639.0,0.034004,NaN,NaN
23723,"[6469, 4900, 21458, 21338, 4110, 2985, 4580, 3...",0.776946,0.714194,0.768828,0.958988,18024.0,0.019481,NaN,NaN
23219,"[34068, 19889, 23098, 30328, 4192, 27915, 4203...",0.764761,0.679330,0.671523,0.952904,16049.0,0.046053,NaN,NaN
7161,"[6420, 34482, 15048, 9393, 8816, 6018, 14039, ...",0.752489,0.651582,0.704055,0.949291,14628.0,0.032426,0.754979,0.502389
...,...,...,...,...,...,...,...,...,...
29804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.725266,0.555282
24155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.781378,0.413372
17181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.700822,0.524335


In [ ]:
analogue_df

In [9]:
analogue_df.describe()

,query_sim,group_sim,sgs,sgs_score,query_id_unsimilar,query_sim_unsimilar,sgs_unsimilar,sgs_score_unsimilar
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,985.000000,985.000000
mean,0.784824,0.735464,0.738398,0.957548,18985.421000,0.033895,0.738481,0.491098
std,0.031377,0.045664,0.031290,0.015605,10818.095266,0.022050,0.031196,0.056167
min,0.716962,0.623999,0.619026,0.896554,26.000000,0.000000,0.619026,0.378090
25%,0.761932,0.701849,0.720571,0.948777,9729.750000,0.020092,0.720640,0.452951
50%,0.781721,0.731256,0.735877,0.957966,18532.000000,0.029506,0.735789,0.484217
75%,0.803512,0.766433,0.760394,0.966872,28568.750000,0.040616,0.760303,0.518952
max,0.876133,0.903892,0.816185,0.996199,37787.000000,0.136147,0.816185,0.790356


In [12]:
analogue_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1979 entries, 35713 to 24344
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   analogue_ids         1000 non-null   object 
 1   query_sim            1000 non-null   float64
 2   group_sim            1000 non-null   float64
 3   sgs                  1000 non-null   float64
 4   sgs_score            1000 non-null   float64
 5   query_id_unsimilar   1000 non-null   float64
 6   query_sim_unsimilar  1000 non-null   float32
 7   sgs_unsimilar        985 non-null    float64
 8   sgs_score_unsimilar  985 non-null    float64
dtypes: float32(1), float64(7), object(1)
memory usage: 211.4+ KB
